In [11]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys
import copy
import matplotlib
%matplotlib inline
sys.path.append("..")
from demo_2_awac import och_2_awac
import adept_envs
import gym

# Load in the data
all_paths = 'demo_list_2elements_PLAYSTYLE_June14.pkl'

paths = pickle.load(open(all_paths, 'rb'))
import itertools

def check_goal_completion(curr_pos):
    max_objs = np.array([0.17, 0.6])
    min_objs = np.array([0.08, 0.2])
    init_bitflips = np.array([0, 0])
    curr_bitflips = init_bitflips.copy()
    for j, pos_idx in enumerate([0, 2]):
        if curr_pos[pos_idx] > max_objs[j]:
            curr_bitflips[j] = 1
        elif curr_pos[pos_idx] < min_objs[j]:
            curr_bitflips[j] = 0
    new_idx = 2*curr_bitflips[0] + curr_bitflips[1]
    return new_idx


input_x = []
for path in paths:
    input_x.append(path['observations'][:-100])
input_x = np.concatenate(input_x, axis=0)[:, 2:6]

input_y = []
start_idxs = []
end_idxs = []
# Get labels
prev_idx = -1
curr_idx = -1
window = 5
state_final_goal_nextgoal_tuples = []

continuous_paths = []
curr_continuous = []
idx_vals = [[] for _ in range(4)]
for j, path in enumerate(paths):
    curr_idx = check_goal_completion(path['observations'][0, 2:6])
    if curr_idx != prev_idx and prev_idx != -1:
        continuous_paths.append(curr_continuous)
        curr_continuous = []
    curr_continuous.append(path)
    prev_idx = check_goal_completion(path['observations'][-1, 2:6])    
    idx_vals[prev_idx].append(path['observations'][-1, 2:6])
continuous_paths.append(curr_continuous)

In [12]:
for path in continuous_paths[0]:
    print(check_goal_completion(path['observations'][0, 2:6]))

3
1
0
2
3
2
0
2
3
1
0
2
3
2
0
1
3
2
0
1
3
2
0
1
0
2
0
1
0


In [13]:
delta_skip = 3
state_nextgoal_finalgoal_tuples = []
for curr_continuous in continuous_paths:
    for j, path in enumerate(curr_continuous):
        leftover_skips = min(len(curr_continuous) - j, delta_skip)
        for k in range(len(path['observations']) - 100):
            for skips in range(leftover_skips):
                curr_ng = check_goal_completion(path['observations'][-1, 2:6]) 
                final_ng = check_goal_completion(curr_continuous[j + skips]['observations'][-1, 2:6]) 
                snf_tuple = (path['observations'][k][:9], curr_ng, final_ng)
                state_nextgoal_finalgoal_tuples.append(snf_tuple)

In [14]:
state_nextgoal_finalgoal_tuples

[(array([ 0.0399838 ,  0.03999283,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.2206686 ,  0.06280294,  2.60419152]),
  1,
  1),
 (array([ 0.0399838 ,  0.03999283,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.2206686 ,  0.06280294,  2.60419152]),
  1,
  0),
 (array([ 0.0399838 ,  0.03999283,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.2206686 ,  0.06280294,  2.60419152]),
  1,
  2),
 (array([ 0.03999389,  0.03999096,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.22225131,  0.05912489,  2.60446963]),
  1,
  1),
 (array([ 0.03999389,  0.03999096,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.22225131,  0.05912489,  2.60446963]),
  1,
  0),
 (array([ 0.03999389,  0.03999096,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.22225131,  0.05912489,  2.60446963]),
  1,
  2),
 (array([ 0.03997692,  0.04000119,  0.22874171,  0.        ,  1.02791483,
          0.        , -0.21640639,  0.

In [15]:
input_x = []
for snf_tuple in state_nextgoal_finalgoal_tuples:
    input_x.append([np.concatenate([snf_tuple[0][2:6], [snf_tuple[2]]])])
input_x = np.concatenate(input_x, axis=0)

input_y = []
for snf_tuple in state_nextgoal_finalgoal_tuples:
    input_y.append(snf_tuple[1])
input_y = np.array(input_y)

In [16]:
input_x.shape

(21017, 5)

In [17]:
from torch.utils.data import TensorDataset, DataLoader
from rlkit.torch.networks import ConcatMlp, Mlp
import torch
import torch.nn as nn
input_size = 5
num_goals = 4
batch_size = 32


goal_predictor = Mlp(
    input_size=input_size,
    output_size=num_goals,
    hidden_sizes=(256, 256, 256),
)

optimizer = torch.optim.Adam(goal_predictor.parameters())

idxs_data = np.array(range(len(input_x)))
np.random.shuffle(idxs_data)
num_train = int(0.9*len(idxs_data))

input_x_train = input_x[idxs_data[:num_train]]
input_x_test = input_x[idxs_data[num_train:]]

input_y_train = input_y[idxs_data[:num_train]]
input_y_test = input_y[idxs_data[num_train:]]

# TODO: Check the syntax
train_ds = TensorDataset(torch.Tensor(input_x_train), torch.Tensor(input_y_train).long())
test_ds = TensorDataset(torch.Tensor(input_x_test), torch.Tensor(input_y_test).long())

trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()

In [18]:
num_epochs = 10
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = goal_predictor(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.8f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            outputs = goal_predictor(torch.Tensor(input_x_test))
            accuracy = np.sum(np.argmax(outputs.detach().numpy(), axis=1) == np.array(input_y_test, dtype=np.int32))/len(input_x_test)
#             print('TEST %f \n'%accuracy*100)
            

print('Finished Training')

[1,     1] loss: 0.00069316
[1,   101] loss: 0.05155905
[1,   201] loss: 0.03240223
[1,   301] loss: 0.03151211
[1,   401] loss: 0.03118895
[1,   501] loss: 0.03079474
[2,     1] loss: 0.00029483
[2,   101] loss: 0.02979413
[2,   201] loss: 0.02879708
[2,   301] loss: 0.02975030
[2,   401] loss: 0.02909476
[2,   501] loss: 0.02941990
[3,     1] loss: 0.00024827
[3,   101] loss: 0.02905188
[3,   201] loss: 0.02846648
[3,   301] loss: 0.02945302
[3,   401] loss: 0.02877927
[3,   501] loss: 0.02736870
[4,     1] loss: 0.00025938
[4,   101] loss: 0.02827456
[4,   201] loss: 0.02816918
[4,   301] loss: 0.02811921
[4,   401] loss: 0.02830851
[4,   501] loss: 0.02840665
[5,     1] loss: 0.00030301
[5,   101] loss: 0.02772153
[5,   201] loss: 0.02871146
[5,   301] loss: 0.02815089
[5,   401] loss: 0.02743869
[5,   501] loss: 0.02719965
[6,     1] loss: 0.00028965
[6,   101] loss: 0.02793885
[6,   201] loss: 0.02850692
[6,   301] loss: 0.02806972
[6,   401] loss: 0.02727113
[6,   501] loss: 0.0

In [19]:
start = 3
curr_s = idx_vals[start][np.random.randint(len(idx_vals[start]))]
curr_g = 1

pl = []
print("STARTING AT %d"%start)
for k in range(100):
    start = 1
    curr_s = idx_vals[start][np.random.randint(len(idx_vals[start]))]
    curr_g = 2
    print("============")
    for j in range(5):
        o = np.concatenate([curr_s, [curr_g]])
        o = torch.Tensor(o)
        prediction = torch.nn.Softmax()(goal_predictor(o)).detach().numpy()
        next_idx = np.random.choice(range(4), p =prediction)
        print("GOING TO %d"%next_idx)
        curr_s = idx_vals[next_idx][np.random.randint(len(idx_vals[next_idx]))]
        if next_idx == curr_g:
            break
    pl.append(j + 1)
np.array(pl).mean()

STARTING AT 3
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING T

/usr/local/google/home/abhishekunique/anaconda3/envs/softlearning-sim-new/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 0
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 2
GOING TO 0
GOING TO 1
GOING TO 3
GOING TO 2
GOING TO 3
GOING TO 1
GOING TO 3
GOING TO 1
GOING TO 0
GOING TO 0
GOING TO 2
GOING TO 0
GOING TO 2
GOING TO 0

3.2

In [15]:
torch.save(goal_predictor, 'BC_model_2elems_window3.pkl')

In [16]:
dat = torch.load('BC_model_2elems_window3.pkl')

In [17]:
goal_predictor.load_state_dict(dat.state_dict())

<All keys matched successfully>